**Introduction**

Video games makes up a major part of most people's entertainment time, attracting players of all ages and backgrounds. For researchers and developers, understanding how different players engage with games can inform better design decisions and improve marketing strategies. In this project, I will work with data collected from a Minecraft server managed by a UBC Computer Science research group led by Frank Wood. This server tracks player activity to support further research in topics such as player behavior. My focus is to explore whether player age and game experience can help predict who is likely to be a high contributor, someone who plays for more than one hour, so that recruitment efforts can be more effective and data quality can be maximized. 

Question: How does age and game experience predict whether a player will be a high contributor (i.e., someone who plays more than one hour)?

Players who contribute more than an hour of gameplay provide more data, which is especially useful for analyzing in-game behavior and training models. By understanding which characteristics are linked to longer playtime, the research team can focus recruitment on players who are more likely to generate meaningful data, helping improve the efficiency and impact of the project.

Dataset description: 
The dataset contains two files: 
1. players.csv file: contain 196 players and 7 variables
+ experience: Experience level of the player (Categorical: Pro, Veteran, Regular, Amateur, Beginner)
+ subscribe: Whether the player subscribed to a newsletter (Logical: True or False)
+ hashedEmail: An unique identifier for each player, used to protect privacy (Categorical)
+ played_hours: Total number of hours a player played (Character)
+ name: Player's username (Character)
+ gender: Player's self-identified gender (Categorical: Male, Female, Non-binary, etc)
+ age: Age of the player in years (Numeric)

2. sessions.csv file: Recording of 1535 game sessions and 5 variables
+ hasedEmail: An unique identifier for each player, used to protect privacy (Categorical)
+ start_time / end_time: Start and end timestamps of game sessions
+ original_start_time / original_end_time: Unix timestamps

Potential issues: 
Several variables in the dataset, such as experience level, are self-reported, which introduces potential issues like inaccuracy, subjectivity, and sampling bias. Players may overestimate their experience, and categories like experience level are subjective with no set definitions. These factors can introduce reduce the predictive accuracy of models, especially when trying to identify high contributors. Careful data cleaning and grouping of sparse categories may help mitigate these limitations.

To answer the question, I will use the players dataset because it contains all the necessary information such as the player characteristics. 

player csv link https://raw.githubusercontent.com/ashleyan1207/DSCI-100-Project/refs/heads/main/players.csv

sessions csv link https://raw.githubusercontent.com/ashleyan1207/DSCI-100-Project/refs/heads/main/sessions.csv

**Methods and Results**

*Loading Data and Wrangling / Cleaning Data*

For this analysis, players are categorized into two contribution levels based on total playtime: High contributors (≥1 hours), who provided most of gameplay data, and low contributors (<1 hour), who contributed less. the one hour threshold was chosen because longer playtime is more likely to produce sufficient data for analysis and model traning. This threshold is further supported by the distribution of playtime, which shows that most players either play for below an hour or well over one hour, making it a natural dividing point for classification. Simplifying the outcome variable into two categories support the use of straightforward classification models, improving interpretability and reducing complexity during model development. 

To do this, I loaded the necessary libraries and cleaned the data. The experience variable is converted into a factor for further classification purposes. A new column, contributor_type, is created to categorize players based on total playtime, aligning with the research question. Finally, rows with missing values are removed to ensure clean and reliable data for analysis. 

In [6]:
library(tidyverse)
library(tidymodels)
library(dplyr)
library(themis)
library(cowplot)
options(repr.matrix.max.rows = 5)

players_url <- "https://raw.githubusercontent.com/ashleyan1207/DSCI-100-Project/refs/heads/main/players.csv"
sessions_url <- "https://raw.githubusercontent.com/ashleyan1207/DSCI-100-Project/refs/heads/main/sessions.csv"

players <- read_csv (players_url) 
sessions <- read_csv (sessions_url) 

players <- players |> 
            mutate(experience = factor (experience, levels = c("Beginner", "Amateur", "Regular", "Pro", "Veteran")), 
            contributor_type = case_when(played_hours >= 1 ~ "High", 
                                         played_hours < 1 ~ "Low")) |> 
            mutate(contributor_type = factor (contributor_type)) |> 
            drop_na ()
cat("Figure 1: Mutated Dataset") 
players

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Figure 1: Mutated Dataset

experience,subscribe,hashedEmail,played_hours,name,gender,Age,contributor_type
<fct>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<fct>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,High
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,High
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,Low
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,17,Low
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17,High


*Summarizing Dataset*

I summarized the dataset by calculating the average playtime, average age, and player count for each experience level. Then, I separately summarized the average age and count of players by contribution type to explore potential patterns in engagement. 

From the summarized dataset, we can deduce that Regular players have the highest average playtime (~18.7 hours), suggesting strong engagement, although majority of players seem to have an Amateur experience level (Fig 2). This suggests that self-reported experience doesn't always reflect actual engagement. Moreover, there is a large imbalance between High (42 players) and Low contributors (152 players), indicating that when performing classification prediction, scaling of the data is needed. 

In [2]:
avg_experience <- players |> 
            group_by(experience) |> 
            summarize(average_played_hours = mean(played_hours, na.rm = TRUE), 
                      average_age = mean(Age), 
                      count = n())

contributors <- players |> 
            group_by(contributor_type) |> 
            summarize(count = n(), average_age = mean(Age)) 

cat("Figure 2: Summarized Player Dataset") 
avg_experience
cat("Figure 3: Summarized Contributor Information") 
contributors 

Figure 2: Summarized Player Dataset

experience,average_played_hours,average_age,count
<fct>,<dbl>,<dbl>,<int>
Beginner,1.2485714,21.65714,35
Amateur,6.0174603,20.25397,63
Regular,18.7257143,20.60000,35
Pro,2.7846154,16.92308,13
Veteran,0.6479167,20.95833,48


Figure 3: Summarized Contributor Information

contributor_type,count,average_age
<fct>,<int>,<dbl>
High,42,20.30952
Low,152,20.57895
